In [7]:
from bs4 import BeautifulSoup
from pprint import pprint
import requests
from selenium import webdriver
import pymysql
import re
import json
from urllib.parse import urlparse, parse_qs

# mysql db connect
# need to fill below host, user, password
#db = pymysql.connect(host='', user='', password='', port=3306, db='readers', charset='utf8', cursorclass=pymysql.cursors.DictCursor)
cursor1 = db.cursor()
cursor2 = db.cursor()
cursor3 = db.cursor()
cursor4 = db.cursor()

#웹 페이지를 열고 소스코드를 읽어오는 작업
def readHtml(url):
    # print(url)
    html = requests.get(url)
    soup = BeautifulSoup(html.text, 'html.parser')
    html.close()
    return soup


In [143]:
# 연재 중 웹툰 크롤링

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"
weekday_sql = "insert ignore into toon_weekday(toon_id, toon_weekday)\
               values (%s, %s)"

weekday = ['mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun']

for day in weekday:
    week_data_url = 'http://webtoon.daum.net/data/pc/webtoon/list_serialized/' + day + '?timeStamp='
    soup = readHtml(week_data_url)
    str_soup = str(soup)
    dict_soup = json.loads(str_soup)
    
    for toon in dict_soup['data']:
        toon_id = 'daum_' + str(toon['id'])
        toon_url = 'http://webtoon.daum.net/data/pc/webtoon/view/' + toon['nickname'] + '?timeStamp'
        toon_name = toon['title']  
        toon_thumb_url = toon['pcThumbnailImage']['url']
        toon_site = 'daum'
        toon_desc = toon['introduction'].replace("  ", " ")
        
        # 완결 여부
        if toon['finishYn'] == 'Y':
            is_end = 'O'
        else:
            is_end = 'X'
        
        # 작가 이름
        wrt_name_list = list()
        for names in toon['cartoon']['artists']:
            wrt_name_list.append(names['name'])
        wrt_name_list = list(set(wrt_name_list))
        wrt_name = '/'.join(wrt_name_list)
        
        # 요일 정보
        for days in toon['webtoonWeeks']:
            cursor1.execute(weekday_sql, (toon_id, days['weekDay']))
        
        # 장르 정보
        for genres in toon['cartoon']['genres']:
            genre_name = genres['name'].replace('순정', '로맨스').replace('코믹', '개그').replace('공포', '스릴러')
            cursor2.execute(genre_sql, (toon_id, genre_name))
        
        # toon_info
        cursor3.execute(sel_sql)
        result = cursor3.fetchall()
        if (toon_id not in result):
            cursor4.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url))
        
        db.commit()
    
db.close()










C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1716-스릴러' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1705-스릴러' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1815-mon' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1815-sat' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1815-개그' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1815-액션' for key 'PRIMARY'")
  self._do_get_result()
C:\Users\KSY\Anaconda3\lib\site-packages\pymysql\curso

In [2]:
# ------ 2. 완결 웹툰 크롤링
# 페이지 넘기고 작품 못 불러오는 거 걍 포기했습니다 누락 작품 700여개 ㅋ

from selenium.common.exceptions import NoSuchElementException

sel_sql = "select toon_id from toon_info"
ins_sql = "insert ignore into toon_info(toon_id, toon_name, toon_url, toon_desc, toon_site, wrt_name, is_end, toon_thumb_url)\
           values (%s, %s, %s, %s, %s, %s, %s, %s)"

#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')
browser.get('http://m.webtoon.daum.net/m/webtoon/category.html#sort=finish')
browser.implicitly_wait(10)
########## PC버전 완결 탭 속 아이템을 도저히 불러올 방법이 없어서 모바일 버전으로 우회함 ㅋ
######### 대신 썸네일 비율이 쓰레기

page = 1
num = 1
while num < 999999:
    try:
        browser.implicitly_wait(10)
        info = browser.find_element_by_xpath("//*[@id=\"daumContent\"]/div[2]/ul["+str(page)+"]/li["+str(num)+"]/a")

        #browser.execute_script("arguments[0].scrollIntoView(true);", webtoon) # Scroll to this webtoon element

        # URL
        toon_url = info.get_attribute('href')
        toon_url = toon_url.replace('m.webtoon', 'webtoon')
        toon_url = toon_url.replace('m/', '')
        
        # ID
        toon_id = 'daum_' + toon_url.replace("http://webtoon.daum.net/webtoon/view/", '')
        
        # 썸네일 주소
        thumb = info.find_element_by_xpath('span[1]/img')
        toon_thumb_url = thumb.get_attribute('data-src')
        
        # 제목
        name = info.find_element_by_class_name('tit')
        toon_name = name.text
        
        # 작가
        name = info.find_element_by_class_name('txt_name')
        wrt_name = name.text
        
        
        toon_desc = ''
        '''
        print(toon_id, toon_name, wrt_name)
        print(toon_url)
        print(toon_thumb_url)
        print("--")
        '''
        # toon_info
        cursor1.execute(sel_sql)
        result = cursor1.fetchall()
        if (toon_id not in result):
            cursor2.execute(ins_sql, (toon_id, toon_name, toon_url, toon_desc, 'daum', wrt_name, 'O', toon_thumb_url))
        db.commit()
        
        num += 1

    except NoSuchElementException:
        if browser.find_element_by_class_name('btn'):
            browser.find_element_by_class_name('btn').click() # 더보기 버튼 클릭
            page +=1
            num = 1
            if page == 50:
                break
            continue
        else:
            print("ERROROROORR")
            break

1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
2 1
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
3 1
3 2
3 3
3 4
3 5
3 6
3 7
3 8
3 9
3 10
3 11
3 12
3 13
3 14
3 15
3 16
3 17
3 18
3 19
3 20
3 21
4 1
4 2
4 3
4 4
4 5
4 6
4 7
4 8
4 9
4 10
4 11
4 12
4 13
4 14
4 15
4 16
4 17
4 18
4 19
4 20
4 21
5 1
5 2
5 3
5 4
5 5
5 6
5 7
5 8
5 9
5 10
5 11
5 12
5 13
5 14
5 15
5 16
5 17
5 18
5 19
5 20
5 21
6 1
6 2
6 3
6 4
6 5
6 6
6 7
6 8
6 9
6 10
6 11
6 12
6 13
6 14
6 15
6 16
6 17
6 18
6 19
6 20
6 21
7 1
7 2
7 3
7 4
7 5
7 6
7 7
7 8
7 9
7 10
7 11
7 12
7 13
7 14
7 15
7 16
7 17
7 18
7 19
7 20
7 21
8 1
8 2
8 3
8 4
8 5
8 6
8 7
8 8
8 9
8 10
8 11
8 12
8 13
8 14
8 15
8 16
8 17
8 18
8 19
8 20
8 21
9 1
9 2
9 3
9 4
9 5
9 6
9 7
9 8
9 9
9 10
9 11
9 12
9 13
9 14
9 15
9 16
9 17
9 18
9 19
9 20
9 21
10 1
10 2
10 3
10 4
10 5
10 6
10 7
10 8
10 9
10 10
10 11
10 12
10 13
10 14
10 15
10 16
10 17
10 18
10 19
10 20
10 21
11 1
11 2
11 3
11 4

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_if1' for key 'PRIMARY'")
  self._do_get_result()


11 15
11 16
11 17
11 18
11 19
11 20
11 21
12 1
12 2
12 3
12 4
12 5
12 6
12 7
12 8
12 9
12 10
12 11
12 12
12 13
12 14
12 15
12 16
12 17
12 18
12 19
12 20
12 21
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1
29 1
30 1
31 1
32 1
33 1
34 1
35 1
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 1


AttributeError: 'NoneType' object has no attribute 'get_attribute'

In [8]:
# ------ 3. 회차 정보
# 연재 작품의 toon_url 수정을 위해 mysql에서 아래 구문 돌리고 실행해야함
# update toon_info set toon_url = replace(toon_url, "/data/pc", "") where toon_site = "daum";

#need to fill below webdriver path
#browser = webdriver.Chrome('C:/.../chromedriver.exe')
sel_url_sql = 'select toon_id, toon_url from toon_info where toon_site = "daum"'
sel_dup_sql = 'select toon_id, epi_name from epi_info where toon_id = %s and epi_name = %s'
ins_epi_info_sql = 'insert into epi_info(toon_id, epi_name, epi_url, epi_thumb_url, epi_date)\
                    values (%s, %s, %s, %s, %s)'

desc_sql = "update toon_info set toon_desc = %s where toon_id = %s and toon_desc = ''"
genre_sql = "insert ignore into toon_genre(toon_id, genre_name)\
             values (%s, %s)"

cursor1.execute(sel_url_sql)
result = cursor1.fetchall()
   
for row in result:
    toon_id = row['toon_id']
    toon_url = row['toon_url']
    
    browser.get(toon_url)
    browser.implicitly_wait(10)
    
    # 성인 작품 다 삭제했지만.. 아마 19금이어서 안 열리는 애 pass
    if "logins.daum.net" in browser.current_url:
        print(toon_id)
        continue
    
    #장르
    gn = browser.find_element_by_class_name("txt_genre")
    tag = 1
    while 1:
        try:
            genres = gn.find_element_by_xpath("a["+ str(tag) +"]")
            genre = genres.text
            genre = genre.replace("#", '')
            tag +=1
            cursor2.execute(genre_sql, (toon_id, genre))
        except: break

    #설명
    desc = browser.find_element_by_class_name("txt_story")
    toon_desc = desc.text
    cursor3.execute(desc_sql,(toon_desc, toon_id))
    
    num = 1 # episode 개수 탐색
    page = 1
    while num < 27:
        try:
            epilist = browser.find_element_by_xpath("//*[@id=\"episodeList\"]/ul/li[" + str(num) + "]")

            episode = epilist.find_element_by_xpath("a")
            #browser.execute_script("arguments[0].scrollIntoView(true);", episode) # Scroll to this episode

            # URL
            epi_url = episode.get_attribute('href')

            # 썸네일 링크
            thumb = episode.find_element_by_class_name('img_thumb')
            epi_thumb_url = thumb.get_attribute('src')

            # 에피소드 제목
            name = episode.find_element_by_class_name("tit_wt")
            epi_name = name.text

            # 업데이트 날짜
            if epilist.get_attribute('class') == "item_preview":
                #date = epilist.find_element_by_class_name('txt_date') # n일 후 무료 이런 식이라서 디비에 안드감ㅋ
                #date = date.text
                date = "2999-12-31"
                # 웹툰 정렬을 위해 이 데이터를 포함한 row는 삭제 필요
            else:
                date = epilist.find_element_by_xpath("div/span")
                if date.get_attribute('class') == "txt_price": # 이거 처리 안해주면 \200 들어감ㅋ
                    date = epilist.find_element_by_xpath("div/span[3]")
                date = date.text
            epi_date = date.replace(".", "-")

            
            cursor2.execute(sel_dup_sql, (toon_id, epi_name))
            if cursor2.fetchall():
                pass
            else:
                cursor3.execute(ins_epi_info_sql, (toon_id, epi_name, epi_url, epi_thumb_url, epi_date))
                db.commit()


            num += 1
            if num == 26:
                page +=1
                browser.get(toon_url + "#pageNo=" + str(page))
                num = 1

        except:
            #print("!!!!!!! EROROR")
            break
        
db.close()

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1047-초능력' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_107-코믹' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_107-모험' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1089-순정' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1089-대학생' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrat

daum_1137


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1185-코믹' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1185-리메이크' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1190-육아' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_11yearafterour-순정' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_11yearafterour-연애' for key 'PRIMARY'")
  self._do_get_result(

daum_1336


c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1339-순정' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1339-재벌' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1342-코믹' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1342-역사' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_1345-추리' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrat

...

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_sadstop-드라마' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_sadstop-이별' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_sadstop-사랑' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_sangod-드라마' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_sangod-판타지' for key 'PRIMARY'")
  self._do_get_result()
c:\u

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_shivering-순정' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_shivering-드라마' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_shivering-사랑' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_shootingstar-판타지' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_shootingstar-순정' for key 'PRIMARY'")
  self._do_

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_threeboys-순정' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_threeboys-드라마' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_threeboys-사랑' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_thrillingstage-드라마' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_thrillingstage-학원' for key 'PRIMARY'")
  self.

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_vacantly-드라마' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_vacantly-가족' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_vacantly-부부' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_villains-드라마' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_villains-액션' for key 'PRIMARY'")
  self._do_get_result(

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_zentlebeast-순정' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_zentlebeast-코믹' for key 'PRIMARY'")
  self._do_get_result()
c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\pymysql\cursors.py:329: Warning: (1062, "Duplicate entry 'daum_zentlebeast-병맛' for key 'PRIMARY'")
  self._do_get_result()
